# Checking Baseline with AutoML

In [1]:
%config InlineBackend.figure_format='retina'
import logging
from ekorpkit import eKonf

logging.basicConfig(level=logging.INFO)
print(eKonf.__version__)

0.1.31+14.g35d2a7e.dirty


In [2]:
data_dir = "../data/fomc"
save_dataframe = eKonf.partial(
    config_group="_func_/save_dataframe", output_dir=data_dir
)
load_dataframe = eKonf.partial(config_group="_func_/load_dataframe", data_dir=data_dir)

## Load a feature set

In [3]:
fs_cfg = eKonf.compose(config_group="dataset=feature")
fs_cfg.name = "fomc_features_small"
fs_cfg.data_dir = data_dir
fs_cfg.data_file = "econ_train_small.parquet"
fs_small = eKonf.instantiate(fs_cfg)

INFO:ekorpkit.datasets.feature:Loaded info file: ../data/fomc/fomc_features_small/info-fomc_features_small.yaml
INFO:ekorpkit.io.file:Loading data from ../data/fomc/fomc_features_small/fomc_features_small-train.parquet
INFO:ekorpkit.info.feature:Added a column [split] with value [train]
INFO:ekorpkit.io.file:Loading data from ../data/fomc/fomc_features_small/fomc_features_small-test.parquet
INFO:ekorpkit.info.feature:Added a column [split] with value [test]


## Auto ML

In [6]:
model_cfg = eKonf.compose(config_group='model/automl=classification')
model_cfg.dataset = fs_cfg
model_cfg.config.time_budget = 30
model_cfg.verbose = False
model = eKonf.instantiate(model_cfg)

INFO:ekorpkit.ekonf:No method defined to call


In [7]:
model.fit()

INFO:ekorpkit.datasets.feature:Loaded info file: ../data/fomc/fomc_features_small/info-fomc_features_small.yaml
INFO:ekorpkit.io.file:Loading data from ../data/fomc/fomc_features_small/fomc_features_small-train.parquet
INFO:ekorpkit.info.feature:Added a column [split] with value [train]
INFO:ekorpkit.io.file:Loading data from ../data/fomc/fomc_features_small/fomc_features_small-test.parquet
INFO:ekorpkit.info.feature:Added a column [split] with value [test]


Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 4, 'num_leaves': 4, 'min_child_samples': 20, 'learning_rate': 0.09999999999999995, 'log_max_bin': 8, 'colsample_bytree': 1.0, 'reg_alpha': 0.0009765625, 'reg_lambda': 1.0}
Best accuracy on validation data: 0.6627
Training duration of best run: 2.4 s


In [8]:
model.best_estimator

LGBMClassifier(learning_rate=0.09999999999999995, max_bin=255, n_estimators=4,
               num_leaves=4, reg_alpha=0.0009765625, reg_lambda=1.0,
               verbose=-1)

In [9]:
model.save()

INFO:ekorpkit.models.automl.flaml:Saved model to /workspace/projects/esgml/outputs/automl_fomc_features_small/automl_fomc_features_small.mdl


In [10]:
model.load()

INFO:ekorpkit.models.automl.flaml:Loaded model from /workspace/projects/esgml/outputs/automl_fomc_features_small/automl_fomc_features_small.mdl


In [11]:
model.show_results()

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 4, 'num_leaves': 4, 'min_child_samples': 20, 'learning_rate': 0.09999999999999995, 'log_max_bin': 8, 'colsample_bytree': 1.0, 'reg_alpha': 0.0009765625, 'reg_lambda': 1.0}
Best accuracy on validation data: 0.6627
Training duration of best run: 2.4 s


In [12]:
model.get_logs()

{'time_history': [10.121073722839355],
 'best_valid_loss_history': [0.33731343283582094],
 'valid_loss_history': [0.33731343283582094],
 'config_history': [{'Current Learner': 'lgbm',
   'Current Sample': 332,
   'Current Hyper-parameters': {'n_estimators': 4,
    'num_leaves': 4,
    'min_child_samples': 20,
    'learning_rate': 0.09999999999999995,
    'log_max_bin': 8,
    'colsample_bytree': 1.0,
    'reg_alpha': 0.0009765625,
    'reg_lambda': 1.0},
   'Best Learner': 'lgbm',
   'Best Hyper-parameters': {'n_estimators': 4,
    'num_leaves': 4,
    'min_child_samples': 20,
    'learning_rate': 0.09999999999999995,
    'log_max_bin': 8,
    'colsample_bytree': 1.0,
    'reg_alpha': 0.0009765625,
    'reg_lambda': 1.0}}],
 'metric_history': [None]}